In [7]:
import gzip
import pickle
from os.path import join, expanduser
import pandas as pd
import json
import numpy as np
from functools import reduce

from article_analysis.parse import get_chunk, get_article_ngram_dict, find_doi_chunk_map, load_ngram_dist
import article_analysis.parse_ent as aape
%load_ext autoreload
%autoreload 1
%aimport article_analysis.parse_ent

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [385]:
head = 50
verbose = True

nlp = aape.init_nlp()

input_path = expanduser('~/data/jstor/latest/')
output_path = expanduser('~/data/jstor/latest/')
prefix = 'ngrams_dict'

with open(join(output_path, 'registry_json.txt')) as file:
    registry_dict = json.loads(file.read())


fname = expanduser('~/data/jstor/latest/corpus_clean_dict.pgz')
with gzip.open(fname) as fp:
    articles_ds = pickle.load(fp)

In [3]:
head = 2
# keywords = ['hypothesis', 'hypotheses', 'table']
# keywords = ['hypothesis', 'Hypothesis', 'hypotheses', 'Hypotheses', 'proposition']
# keywords = ['table']
keywords = ['proposition', 'hypothesis']

In [386]:
nlp.vocab['of'].is_stop

False

In [115]:
all_dois_flat = [v for sublist in registry_dict.values() for v in sublist]
head = 10
head = 2
nst = []

if head > 0:
    all_dois_flat = all_dois_flat[:head]

# check boundaries
batch_size = 100
batch_size = 10
dois_batched = [all_dois_flat[i:i+batch_size] for i in range(0, len(all_dois_flat), batch_size)]

df_agg = []
flags = []
for dois_batch, j in zip(dois_batched, range(len(dois_batched))):
    if verbose:
        print('batch number {0}'.format(j))
    dch_dict = get_article_ngram_dict(dois_batch, registry_dict, input_path, prefix)
    for doi in dois_batch:
        ngram_order = 1
        article_ngrams = dch_dict[doi]
        ngram_positions_list = article_ngrams[ngram_order]
        
        ixs = []
        for keyword in keywords:
            if keyword in ngram_positions_list.keys():
                ixs += ngram_positions_list[keyword]
#         print('doi {0}, len ixs {1}'.format(doi, len(ixs)))
        
        carticle = articles_ds[doi]
        phrases = [phrase for phrase in carticle if any([True for keyword in keywords if keyword in phrase])]
#         print('doi {0}, len ixs {1}'.format(doi, len(phrases)))
#         phrases_start = [phrase for phrase in carticle if any([True for keyword in keywords if keyword == phrase[0]])]
        phrases_start = [phrase for phrase in carticle if any([True for keyword in keywords if keyword == phrase[0]])]

#         print('doi {0}, len ixs {1}; starts {2}.'.format(doi, len(phrases), len(phrases_start)))
        nst.append(len(phrases_start))
        flags.append(len(phrases))
#         for ii in ixs:
#             print(carticle[ii])
# #             print('***')
#             print(carticle[ii+1])
#             phrase = carticle[ii]
#             x = sum([1 for x in phrase if x.isdigit()])
#             phrase2 = carticle[ii+1]
#             x2 = sum([1 for x in phrase2 if x.isdigit()])
#             print(x, len(phrase), x2, len(phrase2))

#         chunks = aape.get_np_candidates(ixs, carticle, nlp, 1)
#         total_counts, total_counts_raw, table, tree_dict = aape.choose_popular_np_phrases(chunks)
#         df = pd.DataFrame(table, columns=['root', 'np', 'count'])
#         df['doi'] = doi
#         df_agg.append(df)

batch number 0


In [563]:
doi = '10.2307/20159684'
carticle = articles_ds[doi]

In [571]:
vc0_, foos = aape.get_high_level_stats(carticle, keywords, nlp)

In [565]:
root2chunk, chunk2root, pos2chunk, chunk2pos, pos2root, root2pos = foos

In [566]:
pos2root[326]

{'consequence', 'mistreatment'}

In [567]:
pos2chunk[326]

{('mistreatment',), ('potential', 'consequence')}

In [562]:
vc0_.head()

,np,np_count.and_ig_ig,root,np_count.andnot_and_ig,np_count.andnot_andnot_and,np_first_occ,root_first_occ
0,"(organizational, policy, grievance)",1.0,grievance,0.0,0.0,0.131868,0.005994
1,"(low, performance, evaluation)",1.0,evaluation,0.0,0.0,0.131868,0.131868
2,"(supervisor, griev, ance)",1.0,ance,0.0,0.0,0.131868,0.130869
3,"(impact,)",1.0,impact,0.0,5.0,0.131868,0.131868
4,"(evalua, tion)",1.0,tion,0.0,0.0,0.131868,0.015984


In [362]:
len(dois_batch)

2

In [366]:
thr_freq = 0.4
thr_len = 10
for doi in dois_batch[:]:
    carticle = articles_ds[doi]
    iis = aape.find_tablelike_phrases(carticle, thr_freq=thr_freq, thr_len=thr_len)
    print(len(iis))
    for i in iis:
        print(carticle[i])
#     stat.sort_values('freq', ascending=False)
#     chunks = aape.get_np_candidates(iis, crticle, nlp, 1)
#     total_counts, total_counts_raw, table, tree_dict = aape.choose_popular_np_phrases(chunks)

#     print(stat.sort_values('freq', ascending=False).head(5))

1
['Because', 'respondents', 'could', 'indicate', 'experience', 'with', 'no', 'orTABLE', '1', 'Comparison', 'of', 'Men', 'and', 'Women', 'on', 'Wall', 'Street', 'by', 'Selected', 'Characteristics', 'Variable', 'All', 'Cases', 'Men', 'Women', 't', '-', 'Background', 'characteristics', 'Married', '60', '70', '54', '1', '.', '40', '.', '06', '.', '09', '.', '08', 'Parent', '43', '56', '34', '1', '.', '76', '.', '06', '.', '10', '.', '08', 'Nonwhite', '10', '4', '15', '1', '.', '63', '.', '04', '.', '04', '.', '06', 'Human', 'capital', 'Economic', 'major', '56', '44', '63', '1', '.', '55', '.', '06', '.', '10', '.', '08', 'Math', 'major', '21', '26', '17', '.', '88', '.', '05', '.', '09', '.', '06', 'Previous', 'Wall', 'St', '.', 'experience', '54', '59', '51', '.', '64', 'with', 'any', '.', '06', '.', '10', '.', '08', 'Job', '-', 'characteristics', 'Hours', 'week', 'mean', '61', '.', '62', '64', '.', '26', '59', '.', '88', '1', '.', '67', '1', '.', '30', '2', '.', '18', '1', '.', '58', 'L

In [369]:
phrase = carticle[iis[-3]]

In [394]:
npb = aape.parse_table_phrases(carticle, iis, nlp)

[['Country', 'of', 'origin']]
[['Gender']]
[['Age']]
[['Collectivism']]
[['Group', 'membership', 'manipulation']]
[['Task', 'manipulation']]
[['Maintenance', 'manipulation']]
[['Reward', 'generosityc'], ['a', 'n']]
[['Values', 'in', 'parentheses', 'are', 'scale', 'reliabilities'], ['b', 'For', 'age', ','], ['years', ','], ['years'], ['c', 'Measured', 'as', 'the', 'sum', 'of', 'Z'], ['p'], ['p'], ['1101TABLE'], ['Results', 'of', 'Regression', 'Analysesa', 'Model'], ['Variableb', 'Model'], ['Individualists', 'Collectivists', 'Model'], ['Collectivists', 'Step'], ['Gender'], ['Age'], ['Country'], ['t'], ['R2'], ['Step'], ['Collectivism'], ['Group', 'membership'], ['t', 'Task', 'inputs'], ['Maintenance', 'inputs'], ['AR2'], ['Step'], ['Collectivism', 'x', 'group', 'membership'], ['Task', 'inputs'], ['Group', 'membership', 'x', 'task', 'inputs'], ['Group', 'membership', 'x', 'maintenance', 'inputs'], ['AR2'], ['Overall', 'model', 'F'], ['Adjusted', 'R2'], ['Standard', 'error'], ['Degrees', '

In [395]:
npb

[('country', ('country',), 154),
 ('origin', ('origin',), 154),
 ('gender', ('gender',), 155),
 ('age', ('age',), 156),
 ('collectivism', ('collectivism',), 157),
 ('manipulation', ('group', 'membership', 'manipulation'), 158),
 ('manipulation', ('task', 'manipulation'), 159),
 ('manipulation', ('maintenance', 'manipulation'), 160),
 ('generosityc', ('reward', 'generosityc'), 161),
 ('value', ('value',), 162),
 ('parenthesis', ('parenthesis',), 162),
 ('reliability', ('scale', 'reliability'), 162),
 ('age', ('age',), 162),
 ('year', ('year',), 162),
 ('year', ('year',), 162),
 ('sum', ('sum',), 162),
 ('z', ('z',), 162),
 ('result', ('result',), 162),
 ('step', ('collectivists', 'step'), 162),
 ('gender', ('gender',), 162),
 ('age', ('age',), 162),
 ('country', ('country',), 162),
 ('step', ('step',), 162),
 ('collectivism', ('collectivism',), 162),
 ('membership', ('group', 'membership'), 162),
 ('input', ('t', 'task', 'input'), 162),
 ('input', ('maintenance', 'input'), 162),
 ('ar2'

In [398]:
vc0_, foos = aape.get_high_level_stats(carticle, keywords, nlp)
iis = aape.find_tablelike_phrases(carticle, thr_freq=thr_freq, thr_len=thr_len)
npb = aape.parse_table_phrases(carticle, iis, nlp)

In [399]:
print(vc0_.shape)
vc0_.head()

(290, 7)


,np,np_count.and_ig_ig,root,np_count.andnot_and_ig,np_count.andnot_andnot_and,np_first_occ,root_first_occ
0,"(hypothesis,)",10.0,hypothesis,0.0,0.0,0.167411,0.167411
1,"(collectivist,)",4.0,collectivist,2.0,24.0,0.008929,0.008929
2,"(individualist,)",2.0,individualist,1.0,14.0,0.008929,0.008929
3,"(median, split)",2.0,split,0.0,2.0,0.363839,0.363839
4,"(collectivism,)",2.0,collectivism,0.0,10.0,0.002232,0.002232


In [400]:
root2chunk, chunk2root, pos2chunk, chunk2pos, pos2root, root2pos = foos

In [433]:
# for root, chunk, pos in npb:
df = pd.DataFrame(npb, columns=['root', 'np_count', 'pos'])
df[['np_count', 'pos']].head()

,np_count,pos
0,"(country,)",154
1,"(origin,)",154
2,"(gender,)",155
3,"(age,)",156
4,"(collectivism,)",157


In [450]:
np_min_pos = dict(df.groupby(['np_count']).apply(lambda x: x.pos.min()))
chunk2root_table = dict(df.set_index(['np_count'])['root'].iteritems())
root_min_pos = dict(df.groupby(['root']).apply(lambda x: x.pos.min()))
vc = pd.DataFrame(df['np_count'].value_counts()).reset_index().rename(columns={'index': 'np'})
vc.head()

,np,np_count
0,"(step,)",3
1,"(age,)",3
2,"(ar2,)",2
3,"(country,)",2
4,"(group, membership)",2


In [455]:
n = max(pos2root.keys())
vc['root'] = vc['np'].apply(lambda x: chunk2root_table[x])
vc['root_first_occ'] = vc['root'].apply(lambda x: min(list(root2pos[x]))/n if x in root2pos.keys() else root_min_pos[x]/n)
vc['np_first_occ'] = vc['np'].apply(lambda x: min(list(chunk2pos[x]))/n if x in chunk2pos.keys() else np_min_pos[x]/n)

In [456]:
vc.head()

,np,np_count,root,root_first_occ,np_first_occ
0,"(step,)",3,step,0.339286,0.361607
1,"(age,)",3,age,0.191964,0.191964
2,"(ar2,)",2,ar2,0.361607,0.361607
3,"(country,)",2,country,0.024554,0.040179
4,"(group, membership)",2,membership,0.075893,0.361607


In [457]:
vc0_, foos = aape.get_high_level_stats(carticle, keywords, nlp)
vc_table = aape.get_high_level_stats_table(carticle, nlp, foos, thr_freq, thr_len)

In [458]:
vc0_.head()

,np,np_count.and_ig_ig,root,np_count.andnot_and_ig,np_count.andnot_andnot_and,np_first_occ,root_first_occ
0,"(hypothesis,)",10.0,hypothesis,0.0,0.0,0.167411,0.167411
1,"(collectivist,)",4.0,collectivist,2.0,24.0,0.008929,0.008929
2,"(individualist,)",2.0,individualist,1.0,14.0,0.008929,0.008929
3,"(median, split)",2.0,split,0.0,2.0,0.363839,0.363839
4,"(collectivism,)",2.0,collectivism,0.0,10.0,0.002232,0.002232


In [496]:
vc_table = vc_table.rename(columns={'np_count': 'np_count.{0}'.format('table')})
non_occ_columns = [c for c in vc_table.columns if 'occ' not in c]
vcs = [vc0_, vc_table[non_occ_columns]]
vcs = reduce(lambda x, y: pd.merge(x, y, left_on=['np', 'root'], right_on=['np', 'root'], how='outer'), vcs)
vcs['np_count.{0}'.format('table')] = vcs['np_count.{0}'.format('table')].fillna(0)

print(vc_table.shape[0], vc0_.shape[0], vcs.shape)

32 290 (296, 8)


In [497]:
# vcs = vcs.set_index('np')

In [498]:
null_mask = vcs['np_first_occ'].isnull()
extra_nps = vcs.loc[null_mask, 'np'].tolist()
# extra_nps = list(vcs.loc[null_mask].index)
extra_nps

[('task', 'manipulation'),
 ('overall', 'model', 'f'),
 ('collectivism', 'x', 'group', 'membership'),
 ('reward', 'generosityc'),
 ('group', 'membership', 'manipulation'),
 ('maintenance', 'manipulation')]

In [499]:
mask_extras = vc_table['np'].isin(extra_nps)
np_occ_dict = dict(vc_table.loc[mask_extras, ['np', 'np_first_occ']].values)
root_occ_dict = dict(vc_table.loc[mask_extras, ['np', 'root_first_occ']].values)

In [500]:
vcs.head()

,np,np_count.and_ig_ig,root,np_count.andnot_and_ig,np_count.andnot_andnot_and,np_first_occ,root_first_occ,np_count.table
0,"(hypothesis,)",10.0,hypothesis,0.0,0.0,0.167411,0.167411,0.0
1,"(collectivist,)",4.0,collectivist,2.0,24.0,0.008929,0.008929,0.0
2,"(individualist,)",2.0,individualist,1.0,14.0,0.008929,0.008929,0.0
3,"(median, split)",2.0,split,0.0,2.0,0.363839,0.363839,0.0
4,"(collectivism,)",2.0,collectivism,0.0,10.0,0.002232,0.002232,2.0


In [511]:
vcs.loc[null_mask, 'np_first_occ'] = vcs.loc[null_mask, 
                                             ['np', 'np_first_occ']].apply(lambda x: np_occ_dict[x['np']], axis=1)
vcs.loc[null_mask, 'root_first_occ'] = vcs.loc[null_mask, 
                                               ['np', 'root_first_occ']].apply(lambda x: np_occ_dict[x['np']], axis=1)

In [513]:
null_mask = vcs['np_first_occ'].isnull()
sum(null_mask)

0

In [538]:
dfr = aape.combine_keyword_table(carticle, keywords, nlp, 0.4, 10)
dfr['doi'] = 'doi:1'
n_phrases = len(carticle)
n_words = sum([len(x) for x in carticle])
dfr['n_phrases'] = n_phrases
dfr['n_words'] = n_words

dfr2 = aape.combine_keyword_table(carticle[:5], keywords, nlp, 0.4, 10)

TypeError: descriptor 'union' of 'set' object needs an argument

In [552]:
dfr2 = aape.combine_keyword_table(carticle[:5], keywords, nlp, 0.4, 10)
dfr2 = pd.DataFrame([[np.nan]*dfr2.shape[1]], columns=dfr2.columns)
dfr2['doi'] = 'doi:2'
n_phrases = len(carticle[:5])
n_words = sum([len(x) for x in carticle[:5]])
dfr2['n_phrases'] = n_phrases
dfr2['n_words'] = n_words


In [553]:
dfr.shape, dfr2.shape

((296, 11), (1, 11))

In [554]:
dfr2.head()

,np_count.and_ig_ig,np_count.andnot_and_ig,np_count.andnot_andnot_and,np_first_occ,root_first_occ,np,np_count.table,root,doi,n_phrases,n_words
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,doi:2,5,103


In [534]:
dfr['doi'] = 'doi:1'
n_phrases = len(carticle)
n_words = sum([len(x) for x in carticle])
dfr['n_phrases'] = n_phrases
dfr['n_words'] = n_words
dfr.head()

,np,np_count.and_ig_ig,root,np_count.andnot_and_ig,np_count.andnot_andnot_and,np_first_occ,root_first_occ,np_count.table,doi,n_phrases,n_words
0,"(hypothesis,)",10.0,hypothesis,0.0,0.0,0.167411,0.167411,0.0,doi:1,449,8609
1,"(collectivist,)",4.0,collectivist,2.0,24.0,0.008929,0.008929,0.0,doi:1,449,8609
2,"(individualist,)",2.0,individualist,1.0,14.0,0.008929,0.008929,0.0,doi:1,449,8609
3,"(median, split)",2.0,split,0.0,2.0,0.363839,0.363839,0.0,doi:1,449,8609
4,"(collectivism,)",2.0,collectivism,0.0,10.0,0.002232,0.002232,2.0,doi:1,449,8609


In [517]:
df_empty = pd.DataFrame()

In [520]:
df_empty['doi'] = 'doi:2'

In [530]:
frames = [dfr, df_empty]

In [523]:
pd.concat([dfr, df_empty], axis=0).head()

/Users/belikov/anaconda/envs/aa/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,doi,np,np_count.and_ig_ig,np_count.andnot_and_ig,np_count.andnot_andnot_and,np_count.table,np_first_occ,root,root_first_occ
0,doi:1,"(hypothesis,)",10.0,0.0,0.0,0.0,0.167411,hypothesis,0.167411
1,doi:1,"(collectivist,)",4.0,2.0,24.0,0.0,0.008929,collectivist,0.008929
2,doi:1,"(individualist,)",2.0,1.0,14.0,0.0,0.008929,individualist,0.008929
3,doi:1,"(median, split)",2.0,0.0,2.0,0.0,0.363839,split,0.363839
4,doi:1,"(collectivism,)",2.0,0.0,10.0,2.0,0.002232,collectivism,0.002232


In [525]:
sorted(dfr.columns, key=lambda x: len(x))

['np',
 'doi',
 'root',
 'np_first_occ',
 'root_first_occ',
 'np_count.table',
 'np_count.and_ig_ig',
 'np_count.andnot_and_ig',
 'np_count.andnot_andnot_and']

In [529]:
len(carticle), sum([len(x) for x in carticle])

(449, 8609)

In [532]:
union_cols = list(set.union(*(set(df.columns) for df in frames)))
sorted(union_cols, key=lambda x: len(x))

['np',
 'doi',
 'root',
 'np_first_occ',
 'root_first_occ',
 'np_count.table',
 'np_count.and_ig_ig',
 'np_count.andnot_and_ig',
 'np_count.andnot_andnot_and']

In [555]:
all_dois_flat[:5]

['10.1525/sop.2004.47.2.189', '10.2307/1556338']